In [1]:
import sys

import torch
import numpy as np

from attacks import FGSMContainer
from basemodels import TorchModelContainer, IrisNN
from datasets import DATASET_LIST, DataContainer, get_dataset_list
# from defences import DefenceContainer

%load_ext autoreload
%autoreload 2

In [2]:
print(sys.version)
print(*sys.path, sep='\n')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/home/lukec/.vscode/extensions/ms-python.python-2020.2.64397/pythonFiles
/home/lukec/.vscode/extensions/ms-python.python-2020.2.64397/pythonFiles/lib/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/venv/lib/python3.6/site-packages
/home/lukec/Downloads/jax/build
/home/lukec/.local/lib/python3.6/site-packages
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/home/lukec/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython


In [3]:
print('Avaliable datasets:')
print(get_dataset_list())

Avaliable datasets:
['MNIST', 'CIFAR10', 'SVHN', 'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed']


In [5]:
DATA_ROOT = 'data'
BATCH_SIZE = 64

# image datasets: {'MNIST', 'CIFAR10', 'SVHN'}
# quantitative datasets: {'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed'}
NAME = 'Iris'
print(f'Starting {NAME} data container...')
print(DATASET_LIST[NAME])

dc = DataContainer(DATASET_LIST[NAME], DATA_ROOT)
dc(size_train=0.8, normalize=True)

num_features = dc.dim_data[0]
num_classes = dc.num_classes
print('Features:', num_features)
print('Classes:', num_classes)

## model in {BCNN, IrisNN, MnistCnnCW}
model = IrisNN(num_features, num_classes, hidden_nodes=16)  # for Iris
model_name = model.__class__.__name__
print('Using model:', model_name)

mc = TorchModelContainer(model, dc)
mc.fit(epochs=200, batch_size=BATCH_SIZE)

Starting Iris data container...
{'name': 'Iris', 'type': 'quantitative', 'size': 150, 'num_classes': 3, 'dim_data': (4,)}
Loading data...
Preparing DataFrame...
Reading from data/iris.data
Spliting train/test sets into numpy arrays...
Successfully load data! Time taken:  0m 0.0s
Features: 4
Classes: 3
Using model: IrisNN
Using device: cuda:0
{'lr': 0.01, 'momentum': 0.9}
[ 1/200]  0m 0.0s - Train Loss: 1.0941 Acc: 32.5000% - Test Loss: 1.0996 Acc: 40.0000%
[ 2/200]  0m 0.0s - Train Loss: 1.0936 Acc: 32.5000% - Test Loss: 1.0993 Acc: 40.0000%
[ 3/200]  0m 0.0s - Train Loss: 1.0925 Acc: 32.5000% - Test Loss: 1.0987 Acc: 40.0000%
[ 4/200]  0m 0.0s - Train Loss: 1.0909 Acc: 34.1667% - Test Loss: 1.0977 Acc: 43.3333%
[ 5/200]  0m 0.0s - Train Loss: 1.0892 Acc: 40.0000% - Test Loss: 1.0966 Acc: 50.0000%
[ 6/200]  0m 0.0s - Train Loss: 1.0874 Acc: 50.0000% - Test Loss: 1.0954 Acc: 53.3333%
[ 7/200]  0m 0.0s - Train Loss: 1.0853 Acc: 60.0000% - Test Loss: 1.0946 Acc: 56.6667%
[ 8/200]  0m 0.0s

In [6]:
attack = FGSMContainer(mc)

In [12]:
adv, y_adv, x_clean, y_clean = attack.generate(count=100)
print(adv.shape)
print(y_adv.shape)
print(x_clean.shape)
print(y_clean.shape)

Time taken for training 30 adversarial examples:  0m 0.1s
(30, 4)
(30,)
(30, 4)
(30,)


In [14]:
accuracy = mc.evaluate(x_clean, y_clean)
print(f'Accuracy on clean samples: {accuracy*100:.4f}%')
accuracy = mc.evaluate(adv, y_clean)
print(f'Accuracy on adversarial example: {accuracy*100:.4f}%')

Accuracy on clean samples: 100.0000%
Accuracy on adversarial example: 46.6667%
